In [5]:
#Importer les bibliothèques nécessaires 
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select

import urllib
import collections

import datetime 
from tkinter import *
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter.filedialog import askopenfilename

In [6]:
def action():
    #initier les listes
    Contact_tel=[]
    Price = []
    Place = []
    Title = []
    #Offer_description = []
    
    %pprint
    
    #recuperer le site entre dans l'interface
    N = str(entry.get())
    
    #Parsing du site principal par Selenium
    url = N
    driver = webdriver.Chrome(r"C:\Users\chaym\Downloads\chromedriver_win32 (3)\chromedriver.exe")
    driver.get(url)

    #Collecte des liens de toutes les offres contenues dans le site principal
    pages_remaining = True
    links=[]

    for i in range(10): 
        #recuperer les infos du site

        for i in range (36):
            try:
                link = driver.find_element_by_xpath("/html/body/section/div[2]/div/ul/li["+str(i)+"]/div[2]/h2").find_element_by_css_selector('a').get_attribute("href")
                links.append(link)

            except :
                link = False

        for i in range (36):
            try:       
                link = driver.find_element_by_xpath("/html/body/section/div[2]/div/ul/li["+str(i)+"]/div[2]").find_element_by_css_selector('a').get_attribute("href")
                links.append(link)
            except :
                link = False
        try:
            #Checks if there are more pages with links
            next_link = driver.find_element_by_xpath('/html/body/section/div[2]/div/div[3]/a[9]/i')
            next_link.click()
        except :
            rows_remaining = False
            
    #pour ne pas avoir de doublons       
    links=list(set(links))
    
    #recuperer les informations dont on a besoin pour chaque annonce
    data= []

    for index in range(len(links)):
        
        link=links[index]
        driver.get(link)
        try:
            Afficher_contact = driver.find_element_by_xpath('//*[@id="stickyDiv"]/div[2]/div[1]/div')
            Afficher_contact.click()
            Contact_tel = driver.find_element_by_xpath('//*[@id="phoneBtnOpened"]').find_element_by_tag_name("p").text
            Price = driver.find_element_by_xpath('/html/body/section/div[2]/div/div[1]/div[1]/div[1]/h3').text 
            Place = driver.find_element_by_xpath('/html/body/section/div[2]/div/div[1]/h3').text
            Title = driver.find_element_by_xpath('/html/body/section/div[2]/div/div[1]/h1').text

            k=Price.split()
            T=Contact_tel.split()
            new1 = []
            new2 = []
            for i in range(len(T)):
                new2.append(T[i])
            for i in range(len(k)-1):
                new1.append(k[i])
            Price2 = []
            Contact2 = []
            Price2= ''.join(new1)
            Contact2= ''.join(new2)
            Price3 = int(Price2)

            data.append({"Contact_tel":Contact2, "Price":Price3,  "Place": Place, "Title":Title})
        except :
            link = False
    #pour indiquer la date du scraping         
    date = datetime.datetime.now()
    
    #Structuration des données
    global data_df
    data_df = pd.DataFrame(data)
    #Elimination des redondonces
    data_df = data_df.drop_duplicates()
    #Structuration sous format Excel
    data_df.to_excel("./date_"+str(date.day)+"_"+str(date.month)+"_"+str(date.year)+".xlsx")

In [33]:
   
def action3():
    
    #pour ne pas avoir les listes d'une facon 
    %pprint
    
    #recuperer les informations d'une date precedente dans une liste data4
    file = str(x)

    data_old = pd.read_excel( file, dtype={'Contact_tel': np.str , 'Price': np.float64})
    data4 = []
    for j in range(len(data_old)):
        k = data_old.iloc[j,1]
        l=data_old.iloc[j,2]
        data4.append({"Contact_tel":k , "Price":l})
    
    data_olddf = pd.DataFrame(data4)
 

    #recuperer les informations (price, tel)de la date recente dans une liste data5
    file2 = str(y)
    data_recent = pd.read_excel(file2, dtype={'Contact_tel': np.str , 'Price': np.float64})
    data5 = []
    for j in range(len(data_recent)):
        k=data_recent.iloc[j,1]
        l=data_recent.iloc[j,2]
        data5.append({"Contact_tel":k , "Price":l})   
    data_rctdf = pd.DataFrame(data5)

        
    #recuperer juste les numeros sans doublons dans une liste tels, et faire la somme des annonces pour chaque tel

    tels = []
    for j in range(len(data_olddf)):
        k=data_olddf.iloc[j,0]
        tels.append(k)
    
    #eliminer les doublons des numeros de tel
    tels = list(set(tels))
    
    #pour avoir la somme des prix des anonces rct pour chaque numero de tel
    somme2=[]
    for j in range(len(tels)):
        s=0
        for i in range(len(data_rctdf)):
            if data_rctdf.iloc[i,0] == tels[j]:
                s= s + data_rctdf.iloc[i,1]         
        somme2.append({"Contact_tel":tels[j] , "Price":s })
        
    
    somme2_df = pd.DataFrame(somme2)



    #pour avoir la somme des prix des anonces old pour chaque numero de tel
    somme1=[]
    for j in range(len(tels)):
        s=0
        for i in range(len(data_olddf)):
            if data_olddf.iloc[i,0] == tels[j]:
                s= s + data_olddf.iloc[i,1]
        somme1.append({"Contact_tel":tels[j] , "Price":s })
       
    somme1_df = pd.DataFrame(somme1)


    
    #gerer la difference entre le total de la date recente et la date old
    difference=[]
    for j in range(len(somme1_df)):
        d=0
        for i in range(len(somme2_df)):
            if somme2_df.iloc[i,0]==somme1_df.iloc[j,0]:
                d = somme1_df.iloc[j,1] - somme2_df.iloc[i,1]
            else:
                pass
        difference.append({"Contact_tel":somme1_df.iloc[j,0] , "Price":d })

    difference_df = pd.DataFrame(difference)
    difference_df.to_excel('C:/Users/chaym/Downloads/differ.xlsx')


    
    #recuperer les informations de ce qui est declare/ data_interne 
    file3 = str(z)
    data_dcl = pd.read_excel(file3, dtype={'Contact_tel': np.str , 'Price': np.float64})
    dcl = []
    for j in range(len(data_dcl)):
        k=data_dcl.iloc[j,1]
        l=data_dcl.iloc[j,2]
        dcl.append({"Contact_tel":k , "Price":l })
        
    dcl_df = pd.DataFrame(dcl)


    diff_fraud = []
    for j in range(len(dcl_df)):
        for i in range(len(difference_df)):
            if dcl_df.iloc[j,0] == difference_df.iloc[i,0]:    
                k=difference_df.iloc[i,0]
                l=difference_df.iloc[i,1] - dcl_df.iloc[j,1]
        if l != 0 :
            diff_fraud.append({"Contact_tel": k , "Actif-NV": l})
    global erreur
    erreur = pd.DataFrame(diff_fraud)
      
    
    fen2= Tk(className='Detection de fraude')
    fen2.configure(bg='grey')
    
    if len(erreur) != 0 :
        lblalerte = Label(fen2, text = "Alerte",font="Times 15", bg="red", fg="blue")
        lblalerte.place(x=10, y=10)
        text1= Label(fen2,text=erreur,font="Times 15")
        text1.place(x=100, y=60)
        text2= Label(fen2,text="total :"+str(len(erreur))+".", font="Times 15")
        text2.place(x=100, y=10)
    else:
        text1= Label(fen2,text="tout est bien",font="Times 15")
        text1.place(x=10, y=10)
    fen2.geometry("400x300")
    fen2.mainloop()

    
    

In [34]:
fen = tk.Tk(className='Scraping || detection de fraude')

fen.geometry("600x300") 
fen.configure(bg='grey')


lblnombre1 = Label(fen, text = "Entrez le site")
lblnombre1.place(x=50, y=50)
entry = Entry(fen)
entry.configure(bg='light grey')
entry.place(x=200, y=50)

Valider = Button(fen, text = "Importer", command = action)
Valider.configure(bg='light blue')
Valider.place(x=350, y=50)

def action2():
    filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
    global x
    x=filename

    lbl2 = Label(fen, text = x)
    lbl2.configure(bg='white')
    lbl2.place(x=200, y=100)

def action4():
    filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
    global y
    y=filename

    lbl2 = Label(fen, text = y)
    lbl2.configure(bg='white')
    lbl2.place(x=200, y=150)

def action6():
    filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
    global z
    z=filename

    lbl2 = Label(fen, text = z)
    lbl2.configure(bg='white')
    lbl2.place(x=200, y=200)



Valider = ttk.Button(fen, text = "Browse file of old data", command = action2)
Valider.place(x=50, y=100)

Valider = ttk.Button(fen, text = "Browse file of recent data", command = action4)
Valider.place(x=50, y=150)

Valider = ttk.Button(fen, text = "Browse file of data_interne", command = action6)
Valider.place(x=50, y=200)

Valider = Button(fen, text = "Executer", command = action3)
Valider.configure(bg='light blue')
Valider.place(x=50, y=250)

        
fen.mainloop()

Pretty printing has been turned OFF
